In [1]:
import json
import periodictable as pt
from refnx.reflect._app.lipid_leaflet import Lipid
lipids = {}

In [2]:
name = 'h-DMPC'
l = Lipid(name, 'C10H18O8NP', 'C26H54', structure=name + '.png',
          references=['DMPC in the fluid phase: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                      'DMPC in the gel phase: S. Tristram-Nagle et al. (2002) Biophys. J. 83: 3324-3335'])
l.add_condition('fluid30C', 319, 782)
l.add_condition('gel10C', 319, 710)
lipids[name] = l

In [3]:
name = 'd54-DMPC'
l = Lipid(name, 'C10H18O8NP', 'C26D54', structure=name + '.png',
              references=['DMPC in the fluid phase: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                          'DMPC in the gel phase: S. Tristram-Nagle et al. (2002) Biophys. J. 83: 3324-3335'])
l.add_condition('fluid30C', 319, 782)
l.add_condition('gel10C', 319, 710)
lipids[name] = l

In [4]:
name = 'h-DMPG'
l = Lipid(name, 'C8H12O10P1', 'C26H54', structure=name + '.png',
          head_exchangable=2,
          references=['Used fluid phase DMPC for tails volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                      'Used headgroup volume from DPPG: G. Pabst et al. (2007) Biophys. J. 93: 513-525'])
l.add_condition('ambient', 257, 782)
lipids[name] = l

In [5]:
name = 'd54-DMPG'
l = Lipid(name, 'C8H12O10P1', 'C26D54', structure=name + '.png',
          head_exchangable=2,
          references=['Used fluid phase DMPC for tails volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195',
                      'Used headgroup volume from DPPG: G. Pabst et al. (2007) Biophys. J. 93: 513-525'])
l.add_condition('ambient', 257, 782)
lipids[name] = l

In [6]:
name = 'h-POPC'
l = Lipid(name, 'C10H18O8NP', 'C32H64', structure=name + '.png',
          references=['S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 319, 944)
lipids[name] = l

In [7]:
name = 'd31-POPC'
l = Lipid(name, 'C10H18O8NP', 'C32H33D31', structure=name + '.png',
          references=['S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 319, 944)
lipids[name] = l

In [8]:
name = 'd9-POPC'
l = Lipid(name, 'C10H9D9O8NP', 'C32H64', structure=name + '.png',
          references=['S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 319, 944)
lipids[name] = l

In [9]:
name = 'h-POPG'
l = Lipid(name, 'C8H12O10P', 'C32H64', structure=name + '.png',
          head_exchangable=2,
          references=['For obtaining heads volume: G. Pabst et al. (2007) Biophys. J. 93: 513-525',
                      'For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469'])
l.add_condition('ambient', 257, 944)
lipids[name] = l

In [10]:
name = 'd31-POPG'
l = Lipid(name, 'C8H12O10P', 'C32H33D31', structure=name + '.png',
          head_exchangable=2,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: G. Pabst et al. (2007) Biophys. J. 93: 513-525 '])
l.add_condition('ambient', 257, 944)
lipids[name] = l

In [11]:
name = 'h-POPS'
l = Lipid(name, 'C8H11O10NP', 'C32H64', structure=name + '.png',
          head_exchangable=1,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: H.I. Petrache et al. (2004) Biophys. J. 86: 1574-1586'])
l.add_condition('ambient', 244, 944)
lipids[name] = l

In [12]:
name = 'd31-POPS'
l = Lipid(name, 'C8H11O10NP', 'C32H33D31', structure=name + '.png',
          head_exchangable=1,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: H.I. Petrache et al. (2004) Biophys. J. 86: 1574-1586'])
l.add_condition('ambient', 244, 944)
lipids[name] = l

In [13]:
name = 'h-POPE'
l = Lipid(name, 'C7H12O8P', 'C32H64', structure=name + '.png',
          head_exchangable=0,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('ambient', 252, 944)
lipids[name] = l

In [14]:
name = 'd31-POPE'
l = Lipid(name, 'C7H12O8P', 'C32H33D31', structure=name + '.png',
          head_exchangable=0,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('ambient', 252, 944)
lipids[name] = l

In [15]:
name = 'd4-POPE'
l = Lipid(name, 'C7H8D4O8P', 'C32H64', structure=name + '.png',
          head_exchangable=0,
          references=['For obtaining tails volume: S.W. Chiu et al. (1999) Biophys. J. 77: 2462-2469',
                      'For obtaining heads volume: J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('ambient', 252, 944)
lipids[name] = l

In [16]:
name = 'h-DPPC'
l = Lipid(name, 'C10H18O8NP', 'C30H62', structure=name + '.png',
          head_exchangable=0,
          references=['J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('gel20C', 319, 825)
l.add_condition('fluid50C', 319, 913)
lipids[name] = l

In [17]:
name = 'd62-DPPC'
l = Lipid(name, 'C10H18O8NP', 'C30D62', structure=name + '.png',
          head_exchangable=0,
          references=['J.F. Nagle and S. Tristram-Nagle (2000) Biochim. Biophys. Acta: Rev. Biomembr. 1469: 159-195'])
l.add_condition('gel20C', 319, 825)
l.add_condition('fluid50C', 319, 913)
lipids[name] = l

In [18]:
with open('lipids.json', 'w') as f:
    json.dump([lipids[p].__dict__ for p in lipids], f)